In [1]:
from json import dumps, loads
import time, os
import threading, Queue
import ast
import unittest
import requests
import random
import logging
# chainsapce
from chainspacecontract import transaction_to_solution
# from chainspacecontract.examples.surge import contract as surge_contract
from chainspacecontract.examples import surge
# crypto
from chainspacecontract.examples.utils import setup, key_gen, pack
from chainspaceapi import ChainspaceClient

# Setup logging
logging.basicConfig(level=logging.INFO, filename="execution.log", filemode="a+",
                        format="%(asctime)-15s %(levelname)-8s %(message)s")
logging.info("Starting test_surge_bids")

# Setup variables
r = requests.get('http://10.129.6.52:4999/setup.in')
setup_str = r.text
setup_str = setup_str.split('\n')
NUM_SHARDS = int(setup_str[0])
NUM_REPLICAS = int(setup_str[1])
NUM_CLIENTS = int(setup_str[2])
logging.info("NUM_SHARDS: %s, NUM_REPLICAS: %s, NUM_CLIENTS: %s "%(NUM_SHARDS, NUM_REPLICAS, NUM_CLIENTS))
CS_HOST='10.129.6.52'
G = setup()[0]


In [2]:
global_client = ChainspaceClient(host=CS_HOST,port=5000)
init_transaction = surge.init()
init_tokens = init_transaction['transaction']['outputs']
global_client.process_transaction(init_transaction)
client_divs = surge.eq_div(NUM_CLIENTS, NUM_SHARDS)
print client_divs

POST http://10.129.6.52:5000/api/1.0/transaction/process HTTP/1.1
{"transaction": {"inputIDs": [], "methodID": "init", "parameters": [], "outputs": ["{\"type\": \"InitToken\", \"location\": 0}", "{\"type\": \"InitToken\", \"location\": 0}", "{\"type\": \"InitToken\", \"location\": 0}", "{\"type\": \"InitToken\", \"location\": 0}", "{\"type\": \"InitToken\", \"location\": 0}", "{\"type\": \"InitToken\", \"location\": 0}", "{\"type\": \"InitToken\", \"location\": 0}", "{\"type\": \"InitToken\", \"location\": 0}", "{\"type\": \"InitToken\", \"location\": 0}", "{\"type\": \"InitToken\", \"location\": 0}", "{\"type\": \"InitToken\", \"location\": 0}", "{\"type\": \"InitToken\", \"location\": 0}", "{\"type\": \"InitToken\", \"location\": 0}", "{\"type\": \"InitToken\", \"location\": 0}", "{\"type\": \"InitToken\", \"location\": 0}", "{\"type\": \"InitToken\", \"location\": 0}", "{\"type\": \"InitToken\", \"location\": 0}", "{\"type\": \"InitToken\", \"location\": 0}", "{\"type\": \"InitToken

HTTP/1.1 502 Bad Gateway
{u'outcome': u'prepared_t_abort', u'success': u'False'}
[75, 75, 75, 75]


In [3]:
# Create prosumer clients for locations 
clients = []
base_port=5000
sreps = []

for s in range(0,NUM_SHARDS):
    clients.append([])
    sreps.append(surge.SREPClient(host=CS_HOST, port=base_port+s))
    
    for c in range(0,client_divs[s]):
        clients[s].append(surge.SurgeClient(host=CS_HOST, port=base_port+s))

print clients

threads = []
for s in range(0,NUM_SHARDS):
    threads.append([])
    for c in range(0,client_divs[s]):
        client = clients[s][c]
        idx = sum(client_divs[:s]) + c
        threads[s].append(threading.Thread(target=clients[s][c].create_surge_client, args=(init_tokens[idx],)) )

for s in range(0,NUM_SHARDS):
    for c in range(0,client_divs[s]):
        threads[s][c].start()

for s in range(0,NUM_SHARDS):
    for c in range(0,client_divs[s]):
        threads[s][c].join()

[[<chainspacecontract.examples.surge.SurgeClient instance at 0x7f9495b53fc8>, <chainspacecontract.examples.surge.SurgeClient instance at 0x7f9495b53f80>, <chainspacecontract.examples.surge.SurgeClient instance at 0x7f9495b0b098>, <chainspacecontract.examples.surge.SurgeClient instance at 0x7f9495b0b050>, <chainspacecontract.examples.surge.SurgeClient instance at 0x7f9495b0b0e0>, <chainspacecontract.examples.surge.SurgeClient instance at 0x7f9495b0b128>, <chainspacecontract.examples.surge.SurgeClient instance at 0x7f9495b0b170>, <chainspacecontract.examples.surge.SurgeClient instance at 0x7f9495b0b1b8>, <chainspacecontract.examples.surge.SurgeClient instance at 0x7f9495b0b200>, <chainspacecontract.examples.surge.SurgeClient instance at 0x7f9495b0b248>, <chainspacecontract.examples.surge.SurgeClient instance at 0x7f9495b0b290>, <chainspacecontract.examples.surge.SurgeClient instance at 0x7f9495b0b2d8>, <chainspacecontract.examples.surge.SurgeClient instance at 0x7f9495b0b320>, <chainspac

POST http://10.129.6.52:5000/api/1.0/transaction/process HTTP/1.1
{"transaction": {"inputIDs": ["0_a41c37bf82e068933b904d567c61585ea0760ef0db715293178d85b1433506ca"], "methodID": "create_surge_client", "parameters": ["c7220292cd02c9bd03876dac36747a611aa118467442c9b8b580566bb53f4d9d7b7334e6e3", "92c71d002b4c6f540c2176c10a3c27bab3f8adc230720c1e67be97c782c74dd1f4c71d002bbf1bfd5cc6f615e00841ac3029c8c4612cd0c22fca96fd617dad1915"], "outputs": ["{\"type\": \"SurgeClient\", \"location\": 0, \"pub\": \"c7220292cd02c9bd03876dac36747a611aa118467442c9b8b580566bb53f4d9d7b7334e6e3\"}", "{\"type\": \"VoteSlipToken\", \"location\": 0, \"pub\": \"c7220292cd02c9bd03876dac36747a611aa118467442c9b8b580566bb53f4d9d7b7334e6e3\"}", "{\"type\": \"EBToken\", \"location\": 0, \"pub\": \"c7220292cd02c9bd03876dac36747a611aa118467442c9b8b580566bb53f4d9d7b7334e6e3\"}"], "returns": [], "dependencies": [], "referenceInputIDs": [], "contractID": "surge"}, "store": {"0_a41c37bf82e068933b904d567c61585ea0760ef0db715293178

POST http://10.129.6.52:5000/api/1.0/transaction/process HTTP/1.1
POST http://10.129.6.52:5000/api/1.0/transaction/process HTTP/1.1
 {"transaction": {"inputIDs": ["0_7a72b649626153d2dee9328ba5643643095a6ca81045b0ac8bec77308529f728"], "methodID": "create_surge_client", "parameters": ["c7220292cd02c9bd02a59d53bfe57c4cbfcd974f3663876a0aea3a3c37a8f7df4afbb0f9c0", "92c71d002b69b74e363cd1d1ddf06948233aee2b40f3f215ec7430c2b74474b5d9c71d002b378609a9ff384abf76424b4c57071b930660341aa0938f45930985a4"], "outputs": ["{\"type\": \"SurgeClient\", \"location\": 0, \"pub\": \"c7220292cd02c9bd02a59d53bfe57c4cbfcd974f3663876a0aea3a3c37a8f7df4afbb0f9c0\"}", "{\"type\": \"VoteSlipToken\", \"location\": 0, \"pub\": \"c7220292cd02c9bd02a59d53bfe57c4cbfcd974f3663876a0aea3a3c37a8f7df4afbb0f9c0\"}", "{\"type\": \"EBToken\", \"location\": 0, \"pub\": \"c7220292cd02c9bd02a59d53bfe57c4cbfcd974f3663876a0aea3a3c37a8f7df4afbb0f9c0\"}"], "returns": [], "dependencies": [], "referenceInputIDs": [], "contractID": "surge"

POST http://10.129.6.52:5000/api/1.0/transaction/process HTTP/1.1
{"transaction": {"inputIDs": ["0_cd6c104ad709b90a1b729171a97d739e890110a9d7ff30a22551ca3bcee75e53"], "methodID": "create_surge_client", "parameters": ["c7220292cd02c9bd03526fc01699437f79167f93234146f80f06423b524eae82dcb1162bcc", "92c71d002b6a902f00d0496af4ca991ab27cd229c46e246224b18e6ec00aeedb44c71d002b9ea096b6e8b5083d3cf478258bd6b8dc33eecf98486e9b8b31f67cda"], "outputs": ["{\"type\": \"SurgeClient\", \"location\": 0, \"pub\": \"c7220292cd02c9bd03526fc01699437f79167f93234146f80f06423b524eae82dcb1162bcc\"}", "{\"type\": \"VoteSlipToken\", \"location\": 0, \"pub\": \"c7220292cd02c9bd03526fc01699437f79167f93234146f80f06423b524eae82dcb1162bcc\"}", "{\"type\": \"EBToken\", \"location\": 0, \"pub\": \"c7220292cd02c9bd03526fc01699437f79167f93234146f80f06423b524eae82dcb1162bcc\"}"], "returns": [], "dependencies": [], "referenceInputIDs": [], "contractID": "surge"}, "store": {"0_cd6c104ad709b90a1b729171a97d739e890110a9d7ff30a2255

{u'outcome': u'prepared_t_abort', u'success': u'False'}
HTTP/1.1 502 Bad GatewayPOST http://10.129.6.52:5000/api/1.0/transaction/process HTTP/1.1
{"transaction": {"inputIDs": ["0_a7d6b8d7e6ebf95fac2d653dc129f64a9a08a77d31c5f9ac81e56ff3615824ab"], "methodID": "create_surge_client", "parameters": ["c7220292cd02c9bd030021728b926408ec94e60877cd9edb228a6a6dcc668df32f817c3a2e", "92c71d002b3f342a3c14b7c307ea8c88f9755472d16c4fa2578c17e6ff93b373a8c71d002b2353630fc91d06a8fcee26ede248998839ced4b1e7277d37ae23baf7"], "outputs": ["{\"type\": \"SurgeClient\", \"location\": 0, \"pub\": \"c7220292cd02c9bd030021728b926408ec94e60877cd9edb228a6a6dcc668df32f817c3a2e\"}", "{\"type\": \"VoteSlipToken\", \"location\": 0, \"pub\": \"c7220292cd02c9bd030021728b926408ec94e60877cd9edb228a6a6dcc668df32f817c3a2e\"}", "{\"type\": \"EBToken\", \"location\": 0, \"pub\": \"c7220292cd02c9bd030021728b926408ec94e60877cd9edb228a6a6dcc668df32f817c3a2e\"}"], "returns": [], "dependencies": [], "referenceInputIDs": [], "contrac

...

POST http://10.129.6.52:5003/api/1.0/transaction/process HTTP/1.1HTTP/1.1 502 Bad Gateway
{"transaction": {"inputIDs": ["3_b965f22b27f1be786ffe65c86eeb09f6b266775190eab1cb5d3345c72ee4fdcb"], "methodID": "create_surge_client", "parameters": ["c7220292cd02c9bd033ab01b8747032f7d402b91e0aa7dd196c48a62c31070b246ddbb6d84", "92c71d002b6376a7fe067f0244cca810d2d5fc10c6203a5d131e4b341772fb936ac71d002bc8fb390eda8cbfcdb04fb437630237d744917122b6476ff5e089d63f"], "outputs": ["{\"type\": \"SurgeClient\", \"location\": 3, \"pub\": \"c7220292cd02c9bd033ab01b8747032f7d402b91e0aa7dd196c48a62c31070b246ddbb6d84\"}", "{\"type\": \"VoteSlipToken\", \"location\": 3, \"pub\": \"c7220292cd02c9bd033ab01b8747032f7d402b91e0aa7dd196c48a62c31070b246ddbb6d84\"}", "{\"type\": \"EBToken\", \"location\": 3, \"pub\": \"c7220292cd02c9bd033ab01b8747032f7d402b91e0aa7dd196c48a62c31070b246ddbb6d84\"}"], "returns": [], "dependencies": [], "referenceInputIDs": [], "contractID": "surge"}, "store": {"3_b965f22b27f1be786ffe65c86ee

HTTP/1.1 502 Bad GatewayPOST http://10.129.6.52:5003/api/1.0/transaction/process HTTP/1.1
{"transaction": {"inputIDs": ["3_e0ae643de259308264c4a6f5367daa7e4e4721c14dcd9561d9b9b5b598da1911"], "methodID": "create_surge_client", "parameters": ["c7220292cd02c9bd0384d66377daf1ffd3d7a2720cd13837eb10ff580e1ad7ac6cf6cab5fd", "92c71d002bfff2f2e8e5e9ee884ac15a5af898a5053aaef0d61ddfb6aee43304ccc71d002b6a9f86edb077db0f368b53a49e16860cc9c1e4983b76bdef5414da0f"], "outputs": ["{\"type\": \"SurgeClient\", \"location\": 3, \"pub\": \"c7220292cd02c9bd0384d66377daf1ffd3d7a2720cd13837eb10ff580e1ad7ac6cf6cab5fd\"}", "{\"type\": \"VoteSlipToken\", \"location\": 3, \"pub\": \"c7220292cd02c9bd0384d66377daf1ffd3d7a2720cd13837eb10ff580e1ad7ac6cf6cab5fd\"}", "{\"type\": \"EBToken\", \"location\": 3, \"pub\": \"c7220292cd02c9bd0384d66377daf1ffd3d7a2720cd13837eb10ff580e1ad7ac6cf6cab5fd\"}"], "returns": [], "dependencies": [], "referenceInputIDs": [], "contractID": "surge"}, "store": {"3_e0ae643de259308264c4a6f5367

HTTP/1.1 502 Bad Gateway
{u'outcome': u'prepared_t_abort', u'success': u'False'}
POST http://10.129.6.52:5003/api/1.0/transaction/process HTTP/1.1
POST http://10.129.6.52:5003/api/1.0/transaction/process HTTP/1.1
HTTP/1.1 502 Bad GatewayHTTP/1.1 502 Bad GatewayHTTP/1.1 502 Bad Gateway{"transaction": {"inputIDs": ["3_88e8907f8970ee51dd9820462e737ca4f08b61761bcb5311fa76a14092c659fb"], "methodID": "create_surge_client", "parameters": ["c7220292cd02c9bd027b3593c1b9b8899f89dd9d14abf0ab64a5a2fd15955c1d517a6414c6", "92c71d002bfbd0c321b7ddf55b6c498c248892dcf0f16c1e5176e63231c5a39f0ac71d002b33fa9e38cb911e31bbb664c0f2af50c1c27315a205812a90bd764121"], "outputs": ["{\"type\": \"SurgeClient\", \"location\": 3, \"pub\": \"c7220292cd02c9bd027b3593c1b9b8899f89dd9d14abf0ab64a5a2fd15955c1d517a6414c6\"}", "{\"type\": \"VoteSlipToken\", \"location\": 3, \"pub\": \"c7220292cd02c9bd027b3593c1b9b8899f89dd9d14abf0ab64a5a2fd15955c1d517a6414c6\"}", "{\"type\": \"EBToken\", \"location\": 3, \"pub\": \"c7220292cd

{"transaction": {"inputIDs": ["3_2eb55054981715201e1e2f302854645bbb05e8706f137c02df82fa1576aa38a4"], "methodID": "create_surge_client", "parameters": ["c7220292cd02c9bd030c09142282175857737bc62e12e268f77fabaec485cff92325d30928", "92c71d002bb924a8f18f935d7dcdc090982d2489a99ed7f97ea3e68c33ebad05cbc71d002bd0e7c0273115868d3c0d8c1aef92e73c4cbae94863463c961369d22f"], "outputs": ["{\"type\": \"SurgeClient\", \"location\": 3, \"pub\": \"c7220292cd02c9bd030c09142282175857737bc62e12e268f77fabaec485cff92325d30928\"}", "{\"type\": \"VoteSlipToken\", \"location\": 3, \"pub\": \"c7220292cd02c9bd030c09142282175857737bc62e12e268f77fabaec485cff92325d30928\"}", "{\"type\": \"EBToken\", \"location\": 3, \"pub\": \"c7220292cd02c9bd030c09142282175857737bc62e12e268f77fabaec485cff92325d30928\"}"], "returns": [], "dependencies": [], "referenceInputIDs": [], "contractID": "surge"}, "store": {"3_2eb55054981715201e1e2f302854645bbb05e8706f137c02df82fa1576aa38a4": "{\"type\": \"InitToken\", \"location\": 3}"}}
HTT

POST http://10.129.6.52:5003/api/1.0/transaction/process HTTP/1.1HTTP/1.1 502 Bad Gateway
{u'outcome': u'prepared_t_abort', u'success': u'False'}
{"transaction": {"inputIDs": ["3_b73e359d3120cf8db1f0a78db2d6025e8b1fe30a85d193608d9e86891bf9207c"], "methodID": "create_surge_client", "parameters": ["c7220292cd02c9bd03f68bc4164bde1ac6a14de9fce91c3af1cc91fa737bf7cca48a091d06", "92c71d002be65a5720619e97a775094d0d5ea00b2a000c82441ed050a59826e2e1c71d002b348595d6234d5919089c52a5691192d5f00b762cabb58f84f7f6fefc"], "outputs": ["{\"type\": \"SurgeClient\", \"location\": 3, \"pub\": \"c7220292cd02c9bd03f68bc4164bde1ac6a14de9fce91c3af1cc91fa737bf7cca48a091d06\"}", "{\"type\": \"VoteSlipToken\", \"location\": 3, \"pub\": \"c7220292cd02c9bd03f68bc4164bde1ac6a14de9fce91c3af1cc91fa737bf7cca48a091d06\"}", "{\"type\": \"EBToken\", \"location\": 3, \"pub\": \"c7220292cd02c9bd03f68bc4164bde1ac6a14de9fce91c3af1cc91fa737bf7cca48a091d06\"}"], "returns": [], "dependencies": [], "referenceInputIDs": [], "contrac

In [4]:
# Create threads for casting srep vote

threads = []
queues = []

for s in range(0,NUM_SHARDS):
    que = Queue.Queue()
    threads.append([])
    for c in range(0,client_divs[s]):
        t = threading.Thread(target=clients[s][c].cast_srep_vote, args=(sreps[s].pub, que)) 
        threads[s].append(t)
    queues.append(que)

# Run bidding threads
for s in range(0,NUM_SHARDS):
    for c in range(0,client_divs[s]):
        threads[s][c].start()

for s in range(0,NUM_SHARDS):
    for c in range(0,client_divs[s]):
        threads[s][c].join()



POST http://10.129.6.52:5000/api/1.0/transaction/process HTTP/1.1
{"transaction": {"inputIDs": ["0_a6cf4298a8407b6cd6d76e780e530492a66687a1480ecfbe20ba28eddc7332d9"], "methodID": "cast_srep_vote", "parameters": ["92c71d002bb44570cb98fc5865c8bf006609b51e87d880a3fcfd6912a5cc6f4d92c71d002b3c4650947993de6b1208a8bbc43b7c4254e4d12db057603083b762a5"], "outputs": ["{\"granted_to\": \"c7220292cd02c9bd02829669d7ea526ac6445aab7a1001b96d07ccf20b05510b74dc463b77\", \"type\": \"SREPVoteToken\", \"granted_by\": \"c7220292cd02c9bd029c419a83acfcb2dd55c999d7491a747157ab196e2c7dfaa00fe25cbf\", \"location\": 0}", "{\"type\": \"VoteSlipToken\", \"location\": 0, \"pub\": \"c7220292cd02c9bd029c419a83acfcb2dd55c999d7491a747157ab196e2c7dfaa00fe25cbf\"}"], "returns": [], "dependencies": [], "referenceInputIDs": [], "contractID": "surge"}, "store": {"0_a6cf4298a8407b6cd6d76e780e530492a66687a1480ecfbe20ba28eddc7332d9": "{\"type\": \"VoteSlipToken\", \"location\": 0, \"pub\": \"c7220292cd02c9bd029c419a83acfcb2dd55

{"transaction": {"inputIDs": ["0_d08e0ee5632caf48faa925b6f7921e4ec1ade195bbfe5cae49ebb45f91be51c1"], "methodID": "cast_srep_vote", "parameters": ["92c71d002b7f69048c624dc4517386fb6b41f476a354a7951a94627af6fecde2c5c71d002b9ad8c29bbdbfb2e5714e170754429c45d803e5101b08015df747ece8"], "outputs": ["{\"granted_to\": \"c7220292cd02c9bd02829669d7ea526ac6445aab7a1001b96d07ccf20b05510b74dc463b77\", \"type\": \"SREPVoteToken\", \"granted_by\": \"c7220292cd02c9bd02da9a15f0ea4860a4ee295d551309ef9d465c3df2cc0518cafda2b564\", \"location\": 0}", "{\"type\": \"VoteSlipToken\", \"location\": 0, \"pub\": \"c7220292cd02c9bd02da9a15f0ea4860a4ee295d551309ef9d465c3df2cc0518cafda2b564\"}"], "returns": [], "dependencies": [], "referenceInputIDs": [], "contractID": "surge"}, "store": {"0_d08e0ee5632caf48faa925b6f7921e4ec1ade195bbfe5cae49ebb45f91be51c1": "{\"type\": \"VoteSlipToken\", \"location\": 0, \"pub\": \"c7220292cd02c9bd02da9a15f0ea4860a4ee295d551309ef9d465c3df2cc0518cafda2b564\"}"}}
POST http://10.129.6.

POST http://10.129.6.52:5000/api/1.0/transaction/process HTTP/1.1
{"transaction": {"inputIDs": ["0_d8489ec40a88f4427f80361710cc6e8ea0b9e48f251c5f74c507cbe04527dd88"], "methodID": "cast_srep_vote", "parameters": ["92c71d002bceb6b5a78deecc81a37c6ef946fdfbdff619bc20484fa7be4f0fc48bc71d002bfc81e3fbb585113fc0a7a988c2724c61d4af6b50c519bcea4bc005e6"], "outputs": ["{\"granted_to\": \"c7220292cd02c9bd02829669d7ea526ac6445aab7a1001b96d07ccf20b05510b74dc463b77\", \"type\": \"SREPVoteToken\", \"granted_by\": \"c7220292cd02c9bd02a1dbd97ea9ef3bba415b630875b416e5ef3cc7ba4c1be78ad78dec23\", \"location\": 0}", "{\"type\": \"VoteSlipToken\", \"location\": 0, \"pub\": \"c7220292cd02c9bd02a1dbd97ea9ef3bba415b630875b416e5ef3cc7ba4c1be78ad78dec23\"}"], "returns": [], "dependencies": [], "referenceInputIDs": [], "contractID": "surge"}, "store": {"0_d8489ec40a88f4427f80361710cc6e8ea0b9e48f251c5f74c507cbe04527dd88": "{\"type\": \"VoteSlipToken\", \"location\": 0, \"pub\": \"c7220292cd02c9bd02a1dbd97ea9ef3bba41

HTTP/1.1 502 Bad Gateway

{u'outcome': u'prepared_t_abort', u'success': u'False'}
POST http://10.129.6.52:5000/api/1.0/transaction/process HTTP/1.1
{"transaction": {"inputIDs": ["0_b3f691023b31218c0bf6dbd0ed9e754ebe4805fba3658cc2c6f12dd9c2b43ca2"], "methodID": "cast_srep_vote", "parameters": ["92c71d002b1ae99ab9b0244009af15b22e5530e3e02b34829f4c76e76f28dcfb0ec71d002be0ff2c181c45ff8838a086e202652460e53089927dac16a2cc8a8a44"], "outputs": ["{\"granted_to\": \"c7220292cd02c9bd02829669d7ea526ac6445aab7a1001b96d07ccf20b05510b74dc463b77\", \"type\": \"SREPVoteToken\", \"granted_by\": \"c7220292cd02c9bd02201b21682fca834bb7e0efc08895f18020021663fd0000f93a48f9e7\", \"location\": 0}", "{\"type\": \"VoteSlipToken\", \"location\": 0, \"pub\": \"c7220292cd02c9bd02201b21682fca834bb7e0efc08895f18020021663fd0000f93a48f9e7\"}"], "returns": [], "dependencies": [], "referenceInputIDs": [], "contractID": "surge"}, "store": {"0_b3f691023b31218c0bf6dbd0ed9e754ebe4805fba3658cc2c6f12dd9c2b43ca2": "{\"type\": \

HTTP/1.1 502 Bad GatewayHTTP/1.1 502 Bad Gateway

{u'outcome': u'prepared_t_abort', u'success': u'False'}{u'outcome': u'prepared_t_abort', u'success': u'False'} HTTP/1.1 502 Bad Gateway

POST http://10.129.6.52:5000/api/1.0/transaction/process HTTP/1.1 HTTP/1.1 502 Bad GatewayHTTP/1.1 502 Bad Gateway
{u'outcome': u'prepared_t_abort', u'success': u'False'}
{u'outcome': u'prepared_t_abort', u'success': u'False'}


{u'outcome': u'prepared_t_abort', u'success': u'False'}

{"transaction": {"inputIDs": ["0_b8aa126fc179f42f662bc636df834943ba377df543cf5b6e9901936dda5622cc"], "methodID": "cast_srep_vote", "parameters": ["92c71d002b08b1adfd43c6867116afb44addfbbea6604dcaef262be91b7c9fd78cc71d002bfd0a4967126485a6d199f325d0ef1b63fe500d71ecf56fb143e1773f"], "outputs": ["{\"granted_to\": \"c7220292cd02c9bd02829669d7ea526ac6445aab7a1001b96d07ccf20b05510b74dc463b77\", \"type\": \"SREPVoteToken\", \"granted_by\": \"c7220292cd02c9bd02babf616b3ea55f612393bcbee10aa487f74a58d070a1e38e1bdd2866\", \"location\

...

POST http://10.129.6.52:5003/api/1.0/transaction/process HTTP/1.1
 HTTP/1.1 502 Bad Gateway
 HTTP/1.1 502 Bad Gateway{"transaction": {"inputIDs": ["3_2a46dec1663737f29e55dc68f13b13bf8351c91228a7a43548273af28c168274"], "methodID": "cast_srep_vote", "parameters": ["92c71d002bcc50758e445468b40e33659baafa81542aecbc4b742cf4a2b9302efac71d002b73ecb7e5705adfb1cdc2c5791d73376a852d7eeee307659995f2ffe8"], "outputs": ["{\"granted_to\": \"c7220292cd02c9bd02c349161ccf161e483bacab16b63f93b4b291adf38ab315384bb720d6\", \"type\": \"SREPVoteToken\", \"granted_by\": \"c7220292cd02c9bd0384d66377daf1ffd3d7a2720cd13837eb10ff580e1ad7ac6cf6cab5fd\", \"location\": 3}", "{\"type\": \"VoteSlipToken\", \"location\": 3, \"pub\": \"c7220292cd02c9bd0384d66377daf1ffd3d7a2720cd13837eb10ff580e1ad7ac6cf6cab5fd\"}"], "returns": [], "dependencies": [], "referenceInputIDs": [], "contractID": "surge"}, "store": {"3_2a46dec1663737f29e55dc68f13b13bf8351c91228a7a43548273af28c168274": "{\"type\": \"VoteSlipToken\", \"location\":

POST http://10.129.6.52:5003/api/1.0/transaction/process HTTP/1.1
{"transaction": {"inputIDs": ["3_70942363daae6346123a7ec6055c07b67c2464a203e48a7e71098f9994c81ed0"], "methodID": "cast_srep_vote", "parameters": ["92c71d002bd8b3d3dc23b5ad4f7dc2e0c394fd5270c32a1913b4adaefd9da915a4c71d002bc1de9c830b142c7234200490ef6ad9fd6e039c70f40f798f16180dc3"], "outputs": ["{\"granted_to\": \"c7220292cd02c9bd02c349161ccf161e483bacab16b63f93b4b291adf38ab315384bb720d6\", \"type\": \"SREPVoteToken\", \"granted_by\": \"c7220292cd02c9bd02f1bdf819689d6486fe62a0f8bfde24912231959f5cad479c62802f46\", \"location\": 3}", "{\"type\": \"VoteSlipToken\", \"location\": 3, \"pub\": \"c7220292cd02c9bd02f1bdf819689d6486fe62a0f8bfde24912231959f5cad479c62802f46\"}"], "returns": [], "dependencies": [], "referenceInputIDs": [], "contractID": "surge"}, "store": {"3_70942363daae6346123a7ec6055c07b67c2464a203e48a7e71098f9994c81ed0": "{\"type\": \"VoteSlipToken\", \"location\": 3, \"pub\": \"c7220292cd02c9bd02f1bdf819689d6486fe

{u'outcome': u'prepared_t_abort', u'success': u'False'}POST http://10.129.6.52:5003/api/1.0/transaction/process HTTP/1.1

{"transaction": {"inputIDs": ["3_f37242736ec887fafe2e6b1ad03449cc0a484cca796e4ef90128625c2c87bc31"], "methodID": "cast_srep_vote", "parameters": ["92c71d002b379bad0d2bbcff93b16c7283275e6f1f8bfcf7bbda69a53114b9f012c71d002b372f7d8f342e8533552366c4ae6d8527c1b8e8af4c865627ade61cf7"], "outputs": ["{\"granted_to\": \"c7220292cd02c9bd02c349161ccf161e483bacab16b63f93b4b291adf38ab315384bb720d6\", \"type\": \"SREPVoteToken\", \"granted_by\": \"c7220292cd02c9bd0309c1ecbdb452a757b93e2cffda23c914ec984c52bbfc87ec0c915298\", \"location\": 3}", "{\"type\": \"VoteSlipToken\", \"location\": 3, \"pub\": \"c7220292cd02c9bd0309c1ecbdb452a757b93e2cffda23c914ec984c52bbfc87ec0c915298\"}"], "returns": [], "dependencies": [], "referenceInputIDs": [], "contractID": "surge"}, "store": {"3_f37242736ec887fafe2e6b1ad03449cc0a484cca796e4ef90128625c2c87bc31": "{\"type\": \"VoteSlipToken\", \"locati

POST http://10.129.6.52:5003/api/1.0/transaction/process HTTP/1.1
{"transaction": {"inputIDs": ["3_fa3a422ae581a3c2249d6c0371854bef65fbac6b1ac0af071f3ebf054bbd3889"], "methodID": "cast_srep_vote", "parameters": ["92c71d002ba73344956d68cc425a3475f5912606b11f1c19f130098b7e0811cfc1c71d002bedae4138ce51976629e41d136f79e924ce1995ccaa05eef9c60f4da3"], "outputs": ["{\"granted_to\": \"c7220292cd02c9bd02c349161ccf161e483bacab16b63f93b4b291adf38ab315384bb720d6\", \"type\": \"SREPVoteToken\", \"granted_by\": \"c7220292cd02c9bd02097ccfd4bd65acf59dd6e8e4147e863b89c84d4d21a3b9bfb8fe104c\", \"location\": 3}", "{\"type\": \"VoteSlipToken\", \"location\": 3, \"pub\": \"c7220292cd02c9bd02097ccfd4bd65acf59dd6e8e4147e863b89c84d4d21a3b9bfb8fe104c\"}"], "returns": [], "dependencies": [], "referenceInputIDs": [], "contractID": "surge"}, "store": {"3_fa3a422ae581a3c2249d6c0371854bef65fbac6b1ac0af071f3ebf054bbd3889": "{\"type\": \"VoteSlipToken\", \"location\": 3, \"pub\": \"c7220292cd02c9bd02097ccfd4bd65acf59d

HTTP/1.1 502 Bad Gateway
{u'outcome': u'prepared_t_abort', u'success': u'False'}
HTTP/1.1 502 Bad Gateway
{u'outcome': u'prepared_t_abort', u'success': u'False'}
HTTP/1.1 502 Bad Gateway
{u'outcome': u'prepared_t_abort', u'success': u'False'}
HTTP/1.1 502 Bad Gateway
{u'outcome': u'prepared_t_abort', u'success': u'False'}
HTTP/1.1 502 Bad Gateway
{u'outcome': u'prepared_t_abort', u'success': u'False'}
HTTP/1.1 502 Bad Gateway
{u'outcome': u'prepared_t_abort', u'success': u'False'}
HTTP/1.1 502 Bad Gateway
{u'outcome': u'prepared_t_abort', u'success': u'False'}
HTTP/1.1 502 Bad Gateway
{u'outcome': u'prepared_t_abort', u'success': u'False'}
HTTP/1.1 502 Bad Gateway
{u'outcome': u'prepared_t_abort', u'success': u'False'}
HTTP/1.1 502 Bad Gateway
{u'outcome': u'prepared_t_abort', u'success': u'False'}
HTTP/1.1 502 Bad Gateway
{u'outcome': u'prepared_t_abort', u'success': u'False'}
HTTP/1.1 502 Bad Gateway
{u'outcome': u'prepared_t_abort', u'success': u'False'}
HTTP/1.1 502 Bad GatewayHTTP

In [5]:
# Create srep without threading
start = time.time()
for s in range(0,NUM_SHARDS):
    votes = []
    que = queues[s]
    while not que.empty():
#     for i in range(0,2):
        votes.append(que.get())
    client = sreps[s]
    sreps[s].create_srep_client(host=CS_HOST, srep_port=base_port+s, vote_tokens=tuple(votes))

POST http://10.129.6.52:5000/api/1.0/transaction/process HTTP/1.1
{"transaction": {"inputIDs": ["0_7a02dd16ca9c797c8daa313e67c53b859ecf0dd38c51848a7849d2370869a33f", "0_492426cd90e16aad9752ce7526dd82a77a27f16836d50bc30044a1fa7b85d926", "0_90c58b2b6e32781c2b3a6b9f0b4a4f804eebcddc1952b40ac22a4aa2b31cd1d8", "0_bac020340a386d3138a69156a1e389da5c929fec301147c9e8365c1cc972900a", "0_3feab63d7297087232007d3f62c82ac8bd33a5e91acfc0cf5e35fd0ad47625fb", "0_b480a78159f511c9023b26c941325d250e1a0345ca79aa45b1e241d5290ef7ea", "0_83d3436a1a4948b40c26f64f650196537df185f0548bad7b7678a4c6ddd5bcfe", "0_fb75f4ccf0e8d3807929063e16b26ad4b58cd7842e4c92029ffa6b6961b546b6", "0_3d8c851cd6dba0d6d956bd65049960135eaa49045c7d7670008148214505f52e", "0_02f41a3c3b1e42f441f5d5a5e62c68a63b37d935156a491e5af73eeb211db926", "0_c39f39674938241ba89705f06058d01dedd8b36813b3c04804722f627121eeda", "0_7f4b836149f110bed061e45cd5a00998f245941576ba267c782ae3842bb2901b", "0_71fa3b409303e2848a3332d08b3d37d86f733b3d8f41a80fe62593af7863c

HTTP/1.1 502 Bad Gateway
{u'outcome': u'prepared_t_abort', u'success': u'False'}
POST http://10.129.6.52:5001/api/1.0/transaction/process HTTP/1.1
{"transaction": {"inputIDs": ["1_f590b5e7bf149ea6c20007861fa5c6092361ce8a6ef27cae3d1921f8610a0b64", "1_67454fe748dcbd8cc5c147adb7d504fc02fef316ef157d47e029313d185ef1d8", "1_ade5a8dcbb834cc043cc8942f369fd74752c271ce6854624762126fe0724a8b5", "1_ca28e30f10fec7d22ab764b12654a373b1fa33af8f70c40bf857e6f0cec5ff07", "1_2ef7ed2854515beca149ea7a3732a61cdc1fabd466a52ae08e51f06d5dd4fcaf", "1_ac34ad42c7b094e9d0bd88fec04cfb086f4edede95bbbd3db47ac30a89cc4df8", "1_6f0db89ad3e2db10e3e88fe0317d1a312102207adf05c366c87534bc97451572", "1_c9b95352b1a1447fdd5d64a4818fdde060a96227c0a2ab94d8b8cac3a5ed396f", "1_2d8e5b6ebfb14131028efa8ca426a6e626af4896897d0203ea18a86ee9462e6a", "1_71e0af0dba713b90e41a6b0d24039a43c607c9a60311abcfe9ba4f4a7cf69228", "1_3ad9cdd94becc63da566feec20347824344d4eebc454a8c1fb65be9ca503cee8", "1_b925039924c97b4eb69f8026f6979091a6ac6b3c4cb15a9d62

HTTP/1.1 502 Bad Gateway
{u'outcome': u'prepared_t_abort', u'success': u'False'}
POST http://10.129.6.52:5002/api/1.0/transaction/process HTTP/1.1
{"transaction": {"inputIDs": ["2_44f03047086e0ad363453c3e479c1f4a72b32298764b1fa459db60d9a1737879", "2_e3ab5322c3237c5a943d26b957e23f19fe55bbd457eb346de0e7a0c6aaf58ddf", "2_9ebb26b8be32c9057f606cb3a51680d93e551269e9125b05328db97cb4ccabac", "2_e3609b4938c791086d988c8d46ea2509321bc31ab1667bc7fd782b2cc843f037", "2_32d9e768e3e9fba10ff589c71d46cb26268f65254506884710cbe98f46e3c788", "2_d31b084a43f739798b37c523afb3ed51b283a849ed4f3bca895007792cfb21a1", "2_2da3e405cb6b2c59adb29e8e1b3ae90d72fdade5d2a389929abd9920e82bb861", "2_cb5301b7329b01bf44109cba60c5eeb1c41c6638729403a3d67043627f3262e4", "2_db6632bc2383425c999f1e934b4bd36017d3d39a7d0486a60aec1b9a0d60500d", "2_067a9a2b5b269c9a1474aa4c3cabe0fa9f168e945801f5ee8e84c430349f7c39", "2_b60f3494d91f67fec3a319c856e80b73335924722303014d33d7c07d9e9fccf2", "2_560d549ad5c2477309e165678f63dc3a74a487f5cf0ef1fce4

HTTP/1.1 502 Bad Gateway
{u'outcome': u'prepared_t_abort', u'success': u'False'}
POST http://10.129.6.52:5003/api/1.0/transaction/process HTTP/1.1
{"transaction": {"inputIDs": ["3_4e5cda6d12ca1a88cccac8a604502a685c1246705fd944250bd918c74dbe19fd", "3_7d20286535ab22611e6822d09420745974c2d046b724dce9cfa8f0f29cb942a8", "3_db876515214e27bf7298f33f6ead3b3b7df71a8e74d4fefa19f47b0f317dcbec", "3_6ac3f0be03eb18e1f2adc54ea9b27e16103dd0b3f0673ebe1dd8d96f6fd1ee2f", "3_dd71fa1d8c93b5be1ea57efffa47c6a86946ac5dbc991f29bcafe0e53bd730f7", "3_78abd5e17dcff9513719ec540d195fb7dbc140268fbb95f386c196a8a199adf9", "3_6ef174135dbdab26ca25bc51ffa721b6fe256260092c9155a85dd94d6faecd8f", "3_490ca1a0191b75856b9218a5d8f79ac4584e83266dda52146a1de62b61a86f9d", "3_4c5b040da9d8fc0f8fa28968f573f06af9d21adb7d2ea05a1b63c833f2f7b728", "3_8052b16eec59c03d98002174a4f87bfed5c804acc920d28e43cc10c3860918c4", "3_a1d729c57b4999f8cb4daeefccba690fac8bf0c36e5d889c04fe867a61e344ac", "3_840bd534a3b24375f489621d68c70c9ceef914fbd59aaef154

HTTP/1.1 502 Bad Gateway
{u'outcome': u'prepared_t_abort', u'success': u'False'}


In [6]:
# # Create srep using threading
# threads = []
# for s in range(0,NUM_SHARDS):
#     votes = []
#     que = queues[s]
#     while not que.empty():
# #     for i in range(0,2):
#         votes.append(que.get())
#     client = sreps[s]
#     t = threading.Thread(target=sreps[s].create_srep_client, args=(base_port, tuple(votes)) )
#     threads.append(t)
    
# for t in threads:
#     t.start()
    
# for t in threads:
#     t.join()

# end = time.time()
# duration = end-start
# logging.info("Execution took "+ str(duration))
# logging.info("TPS: %s"%(NUM_CLIENTS/duration))

In [7]:
end = time.time()
duration = end-start
logging.info("Execution took "+ str(duration))
logging.info("TPS: %s"%(NUM_SHARDS/duration))

with open('stats.csv','a') as f:
    f.write("%s, %s, %s, %s\n"%(NUM_SHARDS, NUM_REPLICAS, NUM_CLIENTS, NUM_SHARDS/duration))
